In [2]:
import pandas as pd 
import spacy 
import json 
from pandas.io.json import json_normalize

In [47]:
#Goal: We want to predict if a user utterance is a question 
# Input: string (1 - nword)
# Output: 1 - question, 0 - not a question

u1 = "How can I sign up for a credit card?"
u2 = "I have 50 dollars in the bank"

def is_question(text: str) -> bool:
    if text.find("How|who"):
        return True
    else:
        return False
    #return text[-1] == '?'
    if "who" in text:
        return True
    elif

is_question(u2)

SyntaxError: invalid syntax (<ipython-input-47-2fa825324830>, line 16)

In [66]:
# Generate question data -- pos_example
rows = []
neg_examples = []

# download question dataset and put into DataFrame
with open("train_5500.label.txt", "r", encoding="ISO-8859-1") as f:
    i = 0
    for l in f.readlines():
        split_line = l.split()
        rows.append({"text": " ".join(split_line[1:]), "label":1})
pos_example = pd.DataFrame(rows)
pd.DataFrame(rows).to_csv("question_dataset.csv", index = False)

data = pd.read_json("train-v2.0.json")
rows = []

for i in range(0, 15):
    print(data['data'][1]['paragraphs'][i])
    if (data['data'][1]['paragraphs'][i] == None):
        i = i+1
    else:
        list = data['data'][i]['paragraphs'][i]['qas']
    for table in list:
        data_row = table['answers']
        time = table['question']
        for row in data_row:
            row['question'] = time
            rows.append(row)
para = pd.DataFrame(rows)
pd.DataFrame(para.text).to_csv("statement_dataset.csv", index = False)

# read question into all lower case 

# test if who, what, when, where, why, how are in the string
# test if a question mark is in the string 
# 

{'qas': [{'question': "What was Frédéric's nationalities?", 'id': '56cbd2356d243a140015ed66', 'answers': [{'text': 'Polish and French', 'answer_start': 182}], 'is_impossible': False}, {'question': 'In what era was Frédéric active in?', 'id': '56cbd2356d243a140015ed67', 'answers': [{'text': 'Romantic era', 'answer_start': 276}], 'is_impossible': False}, {'question': 'For what instrument did Frédéric write primarily for?', 'id': '56cbd2356d243a140015ed68', 'answers': [{'text': 'solo piano', 'answer_start': 318}], 'is_impossible': False}, {'question': 'In what area was Frédéric born in?', 'id': '56cbd2356d243a140015ed69', 'answers': [{'text': 'Duchy of Warsaw', 'answer_start': 559}], 'is_impossible': False}, {'question': 'At what age did Frédéric depart from Poland?', 'id': '56cbd2356d243a140015ed6a', 'answers': [{'text': '20', 'answer_start': 777}], 'is_impossible': False}, {'question': 'What year was Chopin born?', 'id': '56ce0a3762d2951400fa69d6', 'answers': [{'text': '1810', 'answer_s

In [117]:
import json
import spacy 
from tqdm import tqdm 

nlp = spacy.load("en_core_web_sm")

data = json.load(open("dev-v2.0.json", "r"))
data.keys()
#data['data'][0]['paragraphs'][0]['context']
#data['data'][0]['paragraphs'][0]['qas'][0]['question']

extracted_rows = []
for article in tqdm(data['data']):
    for paragraph in article['paragraphs']:
        for sent in nlp(paragraph['context']).sents:
            extracted_rows.append({"text": sent.text, "label": 0})
        
        for question in paragraph['qas']:
            extracted_rows.append({"text": question['question'], "label": 1})

table = pd.DataFrame(extracted_rows)


100%|██████████| 35/35 [00:42<00:00,  1.22s/it]


In [118]:
table.to_csv("qa.csv")

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

train, test = train_test_split(table, stratify = table['label'], random_state = 75)

tfidf = TfidfVectorizer()
tfidf.fit(table['text'])

X_train = tfidf.transform(train['text'])
X_test = tfidf.transform(test['text'])



In [145]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(X_train, train['label'])
clf.score(X_train, train['label'])

preds = clf.predict(X_test)
print(classification_report(test['label'], preds))

              precision    recall  f1-score   support

           0       0.97      0.32      0.48      1650
           1       0.73      0.99      0.84      2969

    accuracy                           0.75      4619
   macro avg       0.85      0.66      0.66      4619
weighted avg       0.81      0.75      0.71      4619



In [146]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, train['label'])
clf.score(X_train, train['label'])

preds = clf.predict(X_test)
print(classification_report(test['label'], preds))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1650
           1       0.98      0.98      0.98      2969

    accuracy                           0.98      4619
   macro avg       0.98      0.97      0.97      4619
weighted avg       0.98      0.98      0.98      4619



/Users/taylornielsen/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [151]:
i = "that cat is awesome?"

ie = tfidf.transform([i])
clf.predict(ie)

array([0])

In [5]:
from sklearn.feature_extraction import stop_words
 
a = stop_words.ENGLISH_STOP_WORDS
print(len(a))
b = set(['who', 'what', 'when', 'where', 'why', 'how'])
c = a.difference(b)
print(len(c))

318
312


In [113]:
import spacy 
nlp = spacy.load("en_core_web_sm")
p = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."""

for s in nlp(p).sents:
    print(s.text)

The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.
They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.
Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia.
The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


In [100]:
import spacy 
p = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."""
nlp = spacy.load('en_core_web_sm')
type(nlp(p))

for sent in nlp(p).sents:
    neg_examples.append({"text": sent.text, "label":0})
neg = pd.DataFrame(neg_examples)
neg

,text,label
0,The Normans (Norman: Nourmands; French: Norman...,0
1,"They were descended from Norse (""Norman"" comes...",0
2,Through generations of assimilation and mixing...,0
3,The distinct cultural and ethnic identity of t...,0


In [304]:
# statement dataset 
neg_example = []
data = pd.read_json("train-v2.0.json")
data['data']


    

0      {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1      {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2      {'title': 'Sino-Tibetan_relations_during_the_M...
3      {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4      {'title': 'The_Legend_of_Zelda:_Twilight_Princ...
                             ...                        
437    {'title': 'Infection', 'paragraphs': [{'qas': ...
438    {'title': 'Hunting', 'paragraphs': [{'qas': [{...
439    {'title': 'Kathmandu', 'paragraphs': [{'qas': ...
440    {'title': 'Myocardial_infarction', 'paragraphs...
441    {'title': 'Matter', 'paragraphs': [{'qas': [{'...
Name: data, Length: 442, dtype: object

In [38]:
pd.concat([pos, neg])

,text,label
0,How did serfdom develop in and then leave Russ...,1
1,What films featured the character Popeye Doyle ?,1
2,How can I find a list of celebrities ' real na...,1
3,What fowl grabs the spotlight after the Chines...,1
4,What is the full form of .com ?,1
...,...,...
999,Who portrayed `` Rosanne Rosanna-Dana '' on th...,1
0,The Normans (Norman: Nourmands; French: Norman...,0
1,"They were descended from Norse (""Norman"" comes...",0
2,Through generations of assimilation and mixing...,0


In [64]:
# Generate question data -- pos_example
rows = []
neg_examples = []

# download question dataset and put into DataFrame
with open("train_5500.label.txt", "r", encoding="ISO-8859-1") as f:
    i = 0
    for l in f.readlines():
        split_line = l.split()
        rows.append({"text": " ".join(split_line[1:]), "label":1})
pos_example = pd.DataFrame(rows)
for i in range(1, len(rows)):
    rows2 = lower(pos_example['text'][i])
    print(rows2)
print(pos_example)
pd.DataFrame(rows).to_csv("question_dataset.csv", index = False)


NameError: name 'lower' is not defined

In [ ]:
import swifter
import spacy 

nlp = spacy.load('en_core_web_sm')

def clean_str(text: str) -> str:
    return " ".join([tok.lemma_ for tok in nlp(text)])

for i in tqdm(range(0, len(table))):
    table['text'].iloc[i] = clean_str(table['text'].iloc[i])


In [ ]:
# remove punctuation from every third entry and lower all text for all entries
import string
def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch.lower() for ch in text if ch not in exclude)

for i in tqdm(range(0, len(table), 3)):
    table['text'].iloc[i] = remove_punc(table['text'].iloc[i])